## 단어의 표현 방법
---
- 국소(이산(Discrete)) 표현(Local Representation): 비슷한 단어들을 맵핑하여 정의
- 분산(연속Continuous) 표현(Distributed Representation): 단어 근처에 자주 등장하는 단어들로 정의

## Bag of Words
---
#### Bag of Words: 단어의 순서를 고려하지 않고 빈도에 집중해 텍스트 데이터를 수치화하는 표현 방법

In [2]:
from konlpy.tag import Okt

okt = Okt()

def build_bag_of_words(document):
    document = document.replace('.', '')
    tokenized_document = okt.morphs(document)

    word_to_index = {}
    bow = []

    for word in tokenized_document:
        if word not in word_to_index.keys():
            word_to_index[word] = len(word_to_index)
            bow.insert(len(word_to_index) -1, 1)

        else:
            index = word_to_index.get(word)
            bow[index] = bow[index] + 1
    
    return word_to_index, bow

In [4]:
# vocab은 단어 사전, bow는 각 인덱스에 해당하는 단어의 출현 빈도
doc1 = "정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다."
vocab, bow = build_bag_of_words(doc1)
vocab, bow

({'정부': 0,
  '가': 1,
  '발표': 2,
  '하는': 3,
  '물가상승률': 4,
  '과': 5,
  '소비자': 6,
  '느끼는': 7,
  '은': 8,
  '다르다': 9},
 [1, 2, 1, 1, 2, 1, 1, 1, 1, 1])

In [5]:
doc2 = "소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다"

vocab, bow = build_bag_of_words(doc2)

vocab, bow

({'소비자': 0,
  '는': 1,
  '주로': 2,
  '소비': 3,
  '하는': 4,
  '상품': 5,
  '을': 6,
  '기준': 7,
  '으로': 8,
  '물가상승률': 9,
  '느낀다': 10},
 [1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1])

In [7]:
# CountVectorizer클래스로 BoW만들기
# CountVectorizer은 띄어쓰기를 기준으로 토큰화

from sklearn.feature_extraction.text import CountVectorizer

corpus = ['you know I want your love. because I love you.']
vector = CountVectorizer()

print('bow vector: ', vector.fit_transform(corpus).toarray())
print('vocab: ', vector.vocabulary_)

bow vector:  [[1 1 2 1 2 1]]
vocab:  {'you': 4, 'know': 1, 'want': 3, 'your': 5, 'love': 2, 'because': 0}


In [8]:
# 불용어 제거 후 BoW만들기

from nltk.corpus import stopwords

text = ["Family is not an important thing. It's everything"]
vector = CountVectorizer(stop_words=["the", "a", "an", "is", "not"])
print("bow vector: ", vector.fit_transform(text).toarray())
print("vocab: ", vector.vocabulary_)

bow vector:  [[1 1 1 1 1]]
vocab:  {'family': 1, 'important': 2, 'thing': 4, 'it': 3, 'everything': 0}


In [9]:
# CountVectorizer의 불용어 사용
vector_count = CountVectorizer(stop_words="english")
print("CountVectorizer bow: ", vector_count.fit_transform(text).toarray())
print("CountVectorizer vocab: ", vector_count.vocabulary_)
# NLTK의 불용어 사용
stopwords_nltk = stopwords.words("english")
vector_nltk = CountVectorizer(stop_words=stopwords_nltk)
print("nltk bow: ", vector_nltk.fit_transform(text).toarray())
print("nltk vocab: ", vector_nltk.vocabulary_)

CountVectorizer bow:  [[1 1 1]]
CountVectorizer vocab:  {'family': 0, 'important': 1, 'thing': 2}
nltk bow:  [[1 1 1 1]]
nltk vocab:  {'family': 1, 'important': 2, 'thing': 3, 'everything': 0}


## DTM
----
#### 문서 단어 행렬(Document-Term Matrix, DTM)
- BoW를 하나의 행렬로 표현
- 예시)
문서1: 먹고 싶은 사과
문서2: 먹고 싶은 바나나
문서3: 길고 노란 바나나 바나나
문서4: 저는 과일이 좋아요

-|과일이|길고|노란|먹고|바나나|사과|싶은|저는|좋아요
-|-|-|-|-|-|-|-|-|-
문서1|0|0|0|1|0|1|1|0|0
문서2|0|0|0|1|1|0|1|0|0
문서3|0|1|1|0|2|0|0|0|0
문서4|1|0|0|0|0|0|0|1|1

#### 몬서 단어 행렬의 한계
- 희소 표현(Sparse representation)
    - 코퍼스가 방대할수록 문서의 차원이 엄청 늘어남
    - 많은 벡터가 0을 가짐(이를 희소 벡터라고 표현)
    - 희소 벡터는 많은 양의 저장공간과 계산 복잡도를 요구
    - 전처리를 통해 단어 집합의 크기를 줄이는 작업이 필요
- 단순 빈도 수 기반 접근
    - 불용어를 제거하더라도 불필요한 단어 존재(ex: the)
    - the가 많이 나오는 두 문서는 다른 내용이더라도 유사도가 높다고 착각 가능

## TF-IDF
---
#### TF-IDF(단어 빈도-역 문서 빈도, Term Frequency-Inverse Document Frequency)
- 단어의 빈도와 역문서 빈도(문서의 빈도에 특정 식을 취함)를 사용하여 DTM내 단어들에 가중치를 곱해 중요도 부여
- tf(d,t): 특정 문서 d에서의 특정 단어 t의 등장 횟수
- df(t): 특정 단어 t가 등장한 문서의 수
- idf(t): df(t)에 반비례하는 수
    - $idf(t) = log(\displaystyle\frac{n}{1+df(t)})$
    - 로그를 취하는 이유: 
        - 단순 역수만 취한다면 문서의 수가 커진다면 idf값이 기하급수적으로 커지기 때문
        - 빈도가 적은 희귀 단어에 너무 큰 가중치가 부여될 수 있음
- TF-IDF값이 높으면 중요도가 높고 낮으면 중요도가 낮다.

-|과일이|길고|노란|먹고|바나나|사과|싶은|저는|좋아요
-|-|-|-|-|-|-|-|-|-
문서1|0|0|0|1|0|1|1|0|0
문서2|0|0|0|1|1|0|1|0|0
문서3|0|1|1|0|2|0|0|0|0
문서4|1|0|0|0|0|0|0|1|1

위 DTM을 구한 값이 TF가 되고 이 TF에 반비례하는 가중치인 IDF를 곱해주면

단어|IDF(역 문서 빈도)
-|-
과일이|ln(4/(1+1)) = 0.69
길고|ln(4/(1+1)) = 0.69
노란|ln(4/(1+1)) = 0.69
먹고|ln(4/(2+1)) = 0.28
바나나|ln(4/(2+1)) = 0.28
사과|ln(4/(1+1)) = 0.

-|과일이|길고|노란|먹고|바나나|사과|싶은|저는|좋아요
-|-|-|-|-|-|-|-|-|-
문서0.28|0|0|0|0.28|0|0.28|0.28|0|0
문서0.69|0|0|0|0.28|0.28|0|0.28|0|0
문서3|0|0.28|0.28|0|0.57|0|0|0|0
문서4|0.28|0|0|0|0|0|0|0.28|


문서3에서의 바나나의 가중치가 문서2와 달랐던 이유는 문서2에서는 한번 언급했지만 문서3에서는 두번 언급했기 때문에 더 중요한 단어로 판단했기 때문

In [11]:
import pandas as pd
from math import log

docs = [
  '먹고 싶은 사과',
  '먹고 싶은 바나나',
  '길고 노란 바나나 바나나',
  '저는 과일이 좋아요'
] 

vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()

vocab

['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']

In [17]:
N = len(docs)

def tf(t, d):
    return d.count(t)

def idf(t):
    df = 0
    for doc in docs:
        df += t in doc
    return log(N/(df+1))

def tfidf(t, d):
    return tf(t,d)*idf(t)

In [13]:
result = []

for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        result[-1].append(tf(t,d))

tf_ = pd.DataFrame(result, columns=vocab)

tf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


In [18]:
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index=vocab, columns=["IDF"])
idf_

,IDF
과일이,0.693147
길고,0.693147
노란,0.693147
먹고,0.287682
바나나,0.287682
사과,0.693147
싶은,0.287682
저는,0.693147
좋아요,0.693147


In [19]:
result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns=vocab)
tfidf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147
